Generating the list of Mersenne numbers. Mersenne numbers can only be prime if their exponent,  𝑝 , is prime. Make a list of the Mersenne numbers for all primes  𝑝  between 3 and 65 (there should be 17 of them).

In [1]:
# First of all lets define a simple function to detect whether number is prime or not 
def is_prime(number):
  if number<2:
    return False
  else : 
    for i in range(2,number):
      if number%i == 0:
        return False 
  return True

In [2]:
def mersenne_numbers(n_start, n_end):
  mersenne = []
  for i in range(n_start,n_end+1):
    if is_prime(i):
      mersenne.append(2**i-1)
  return mersenne 


In [3]:
# Have a look at how large the numbers get. Now not every number is a prime amongst them as we know that vice-versa is not true. 
# Also reflect upon high tough it is to check these numbers as prime with the conventional methods as numbers are very large. 
mersenne_numbers(3,65)

[7,
 31,
 127,
 2047,
 8191,
 131071,
 524287,
 8388607,
 536870911,
 2147483647,
 137438953471,
 2199023255551,
 8796093022207,
 140737488355327,
 9007199254740991,
 576460752303423487,
 2305843009213693951]

We can test if a Mersenne number is prime using the Lucas-Lehmer test. First let's write a function that generates the sequence used in the test. Given a Mersenne number with exponent $p$, the sequence can be defined as

$$ n_0 = 4 $$
$$ n_i = (n_{i-1}^2 - 2) mod (2^p - 1) $$

Write a function that accepts the exponent $p$ of a Mersenne number and returns the Lucas-Lehmer sequence up to $i = p - 2$ (inclusive). 

In [4]:
def lucas_lehmer(p):
  n_0 = 4
  x = 2**p-1
  lucas_lehmer_series = []
  lucas_lehmer_series.append(n_0)
  for i in range(0,p-2):
    n_0 = (n_0**2-2)%x
    lucas_lehmer_series.append(n_0)
  return lucas_lehmer_series



In [5]:
lucas_lehmer(17)

[4,
 14,
 194,
 37634,
 95799,
 119121,
 66179,
 53645,
 122218,
 126220,
 70490,
 69559,
 99585,
 78221,
 130559,
 0]

In [7]:
print(len(lucas_lehmer(17)))

16


In [8]:
# Now as per lucas lehmer test for mersenne number for p. We should check for p-2 in the lucas_lehmer sequence. 
# If p-2 th index in the lucas lehmer sequence is zero then the mersenne number is a prime 
sequence_lehmer = lucas_lehmer(17)
sequence_lehmer[15]

0

For a given Mersenne number with exponent $p$, the number is prime if the Lucas-Lehmer series is 0 at position $p-2$. Write a function that tests if a Mersenne number with exponent $p$ is prime. Test if the Mersenne numbers with prime $p$ between 3 and 65 (i.e. 3, 5, 7, ..., 61) are prime. Your final answer should be a list of tuples consisting of `(Mersenne exponent, 0)` (or `1`) for each Mersenne number you test, where `0` and `1` are replacements for `False` and `True` respectively.

**HINT:** You may want to use the [`zip`](https://docs.python.org/3/library/functions.html#zip) function which returns an iterable of tuples resulting in a pair-wise combination of two iterables (e.g., two lists).

In [9]:
# It is intersting problem. We need to make two seperate list of p values between 3 and 65 (which are prime)
# In front of them we need to put 1 if at that exponent p the mersenne number is a prime. Simlarly we need to put 0 if at that exponent p 
# mesenne number is not prime 

def is_prime(number):
    if(number<2):
        return False
    else :
        for i in range(2,number-1):
            if number%i == 0:
                return False
        return True

def ll_prime(p):
    result_prime = []
    for i in range(3,p):
        if(is_prime(i)):
            result_prime.append(i)
    return result_prime

def lucas_lehmer(p):
    n_0 = 4
    x = (2**(p)-1)
    ll_sequence = []
    ll_sequence.append(n_0)
    for i in range(1,p-1):
        n_0 = (n_0**(2)-2)% x
        ll_sequence.append(n_0)
    return ll_sequence
        
def check_prime(p):
    ll_sequence = lucas_lehmer(p)
    if ll_sequence[p-2]==0:
        return 1
    else :
        return 0

mersenne_exponent = ll_prime(65)
mersenne_results = []
for i in range(0,17):
    k = check_prime(mersenne_exponent[i])
    mersenne_results.append(k)
result = zip(mersenne_exponent, mersenne_results)
result_set = list(result)
print(result_set)

[(3, 1), (5, 1), (7, 1), (11, 0), (13, 1), (17, 1), (19, 1), (23, 0), (29, 0), (31, 1), (37, 0), (41, 0), (43, 0), (47, 0), (53, 0), (59, 0), (61, 1)]


# Sieve of Eratosthenes

In this we will develop an even faster method which is known as the Sieve of Eratosthenes (although it will be more expensive in terms of memory). The Sieve of Eratosthenes is an example of dynamic programming, where the general idea is to not redo computations we have already done (read more about it here). We will break this sieve down into several small functions.

Our submission will be a list of all prime numbers less than 2000.

The method works as follows (see here for more details)

Generate a list of all numbers between 0 and N; mark the numbers 0 and 1 to be not prime
Starting with  𝑝=2  (the first prime) mark all numbers of the form  𝑛𝑝  where  𝑛>1  and  𝑛𝑝<=𝑁  to be not prime (they can't be prime since they are multiples of 2!)
Find the smallest number greater than  𝑝  which is not marked and set that equal to  𝑝 , then go back to step 2. Stop if there is no unmarked number greater than  𝑝  and less than  𝑁+1 
We will break this up into a few functions, our general strategy will be to use a Python list as our container although we could use other data structures. The index of this list will represent numbers.

We have implemented a sieve function which will find all the prime numbers up to  𝑛 . We will need to implement the functions which it calls. They are as follows

list_true Make a list of true values of length  𝑛+1  where the first two values are false (this corresponds with step 1 of the algorithm above)
mark_false takes a list of booleans and a number  𝑝 . Mark all elements  2𝑝,3𝑝,...𝑛  false (this corresponds with step 2 of the algorithm above)
find_next Find the smallest True element in a list which is greater than some  𝑝  (has index greater than  𝑝  (this corresponds with step 3 of the algorithm above)
prime_from_list Return indices of True values
Remember that python lists are zero indexed. We have provided assertions below to help us assess whether our functions are functioning properly.

In [12]:
def sieve(n):
 
    # Create a boolean array
    # "prime[0..n]" and initialize
    #  all entries it as true.
    # A value in prime[i] will
    # finally be false if i is
    # Not a prime, else true.
    prime_list = []
    prime = [True for i in range(n+1)]
    p = 2
    while (p * p <= n):
 
        # If prime[p] is not
        # changed, then it is a prime
        if (prime[p] == True):
 
            # Update all multiples of p
            for i in range(p * p, n+1, p):
                prime[i] = False
        p += 1
 
    # Print all prime numbers
    for p in range(2, n+1):
        if prime[p]:
            prime_list.append(p)
    return prime_list

In [13]:
sieve(1000)

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199,
 211,
 223,
 227,
 229,
 233,
 239,
 241,
 251,
 257,
 263,
 269,
 271,
 277,
 281,
 283,
 293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509,
 521,
 523,
 541,
 547,
 557,
 563,
 569,
 571,
 577,
 587,
 593,
 599,
 601,
 607,
 613,
 617,
 619,
 631,
 641,
 643,
 647,
 653,
 659,
 661,
 673,
 677,
 683,
 691,
 701,
 709,
 719,
 727,
 733,
 739,
 743,
 751,
 757,
 761,
 769,
 773,
 787,
 797,
 809,
 811,
 821,
 823,
 827,
 829,
 839,
 853,
 857,
 859,
 863,
 877,
 881,
 883,
 887,
 907,
 911,
 919,
 929,
 937,
 941,
 947,
 953,
 967,
 971,
 977,
 983,
 991,
 997]